In [3]:
# universal_model_benchmark.py

import os
import time
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import (
    mean_squared_error, r2_score, roc_auc_score, accuracy_score, log_loss
)
from sklearn.model_selection import cross_val_score

# Load datasets
X_full_df = pd.read_csv("../model_food/data/financial_data_scaled.csv")
y_full = X_full_df["market_stress"]
X_full = X_full_df.drop(columns=["market_stress"])

X_pca_df = pd.read_csv("../model_food/data/financial_data_pca.csv")
y_pca = X_pca_df["market_stress"]
X_pca = X_pca_df.drop(columns=["market_stress"])

X_synth_df = pd.read_csv("../model_food/data/synth_findata.csv")
for col in X_synth_df.select_dtypes(include="object").columns:
    try:
        X_synth_df[col] = pd.to_datetime(X_synth_df[col])
        X_synth_df[col + "_days_since_start"] = (X_synth_df[col] - X_synth_df[col].min()).dt.days
        X_synth_df.drop(columns=[col], inplace=True)
    except:
        X_synth_df.drop(columns=[col], inplace=True)
X_synth_df = X_synth_df.select_dtypes(include=[np.number])
y_synth = X_synth_df["market_stress"]
X_synth = X_synth_df.drop(columns=["market_stress"])

In [5]:
# Benchmarking function
def benchmark_model(name, model_path, X, y, feature_selector=None, use_predict_proba=True, thresholding=False):
    try:
        with open(model_path, "rb") as f:
            model = pickle.load(f)

        if feature_selector and os.path.exists(feature_selector):
            with open(feature_selector, "rb") as f:
                selected_features = pickle.load(f)
            X = X.iloc[:, selected_features]

        start = time.time()
        if use_predict_proba:
            y_proba = model.predict_proba(X)[:, 1]
        elif thresholding:
            y_scores = model.predict(X)
            y_proba = y_scores
        else:
            y_pred = model.predict(X)
            y_proba = y_pred.astype(float)

        y_pred = (y_proba >= 0.5).astype(int)
        elapsed = time.time() - start

        metrics = {
            "Model": name,
            "MSE": mean_squared_error(y, y_proba),
            "R² Score": r2_score(y, y_proba),
            "ROC-AUC Score": roc_auc_score(y, y_proba),
            "Accuracy Score": accuracy_score(y, y_pred),
            "Log Loss": log_loss(y, y_proba),
            "Prediction Time (s)": elapsed,
        }

        try:
            stability = np.mean(cross_val_score(model, X, y, cv=5))
        except:
            stability = np.nan

        metrics["Hotness"] = stability
        metrics["Cleverness"] = (metrics["ROC-AUC Score"] + metrics["Accuracy Score"]) / 2
        metrics["Fitness Score"] = (
            metrics["ROC-AUC Score"] + metrics["Accuracy Score"] + (stability if not np.isnan(stability) else 0)
        ) / (3 if not np.isnan(stability) else 2)
        metrics["Lineage"] = "Universal Benchmark"
        return metrics

    except Exception as e:
        return {"Model": name, "Error": str(e)}

In [7]:
# List of models to benchmark
model_configs = [
    ("CNN (MLP)", "OGOGszz/saved_models/CNN (MLP).pkl", X_full, y_full, "saved_models/CNN_selected_features.pkl"),
    ("Diffusion (GMM)", "OGOGszz/saved_models/Diffusion_Model_GMM.pkl", X_pca, y_pca, None),
    ("Elastic Net", "OGOGszz/saved_models/Elastic Net.pkl", X_full, y_full, None, False, True),
    ("GA-Optimized LR", "OGOGszz/saved_models/GA-Optimized LR.pkl", X_full, y_full, "saved_models/GA_selected_features.pkl"),
    ("Gradient Boosting", "OGOGszz/saved_models/GradientBoosting.pkl", X_full, y_full),
    ("NeuroEvolution (NEAT)", "OGOGszz/saved_models/NeuroEvolution (NEAT).pkl", X_full, y_full),
    ("SGD", "OGOGszz/saved_models/SGD.pkl", X_full, y_full)
]

In [9]:
# Run benchmarks
results = []
for config in model_configs:
    result = benchmark_model(*config)
    results.append(result)

/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GaussianMixture was fitted without feature names
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/sophiaboettcher/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SGDClassifier was fitted without feature names
  warnings.warn(


In [13]:
# Output results
results_df = pd.DataFrame(results)
results_df.to_csv("OGOGszz/perf/model_performance_universal.csv", index=False)
print("✅ Benchmarking complete. Results saved to model_performance_universal.csv")

✅ Benchmarking complete. Results saved to model_performance_universal.csv
